In [291]:
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt

In [292]:
def load_data(fname):
    with open(fname, "rb") as f:
        return pkl.load(f)

In [293]:
def get_val_accs(data, iterations=15):
    val_accs = []
    for seed_data in data:

        iteration_val_accs = []
        for iteration_data in seed_data:
            iteration_val_acc = iteration_data['train']["test_acc"]
            iteration_val_accs.append(iteration_val_acc)
        val_accs.append(iteration_val_accs)
    return np.array(val_accs)


def get_neurons(data, iterations=15):
    neurons = []
    for seed_data in data:

        iteration_neurons_l0 = []
        iteration_neurons_l1 = []
        for iteration_data in seed_data:
            iteration_neuron = iteration_data['prune'][0]["neurons"]
            iteration_neurons_l0.append(iteration_neuron)
            iteration_neuron = iteration_data['prune'][1]["neurons"]
            iteration_neurons_l1.append(iteration_neuron)
        neurons.append((iteration_neurons_l0, iteration_neurons_l1))
    return neurons


def get_ckas(data, iterations=15):
    ckas = []
    for seed_data in data:

        iteration_ckas = []
        for iteration_data in seed_data:
            iteration_cka = min(x["ckas"][-1] for x in iteration_data['prune'])
            iteration_ckas.append(iteration_cka)
        ckas.append(iteration_ckas)
    return np.array(ckas)

In [294]:
cka_data = load_data(f"_in-784_hd-512-512_out-10_dr-0.5_strategy-iterative_type-cka.pkl")
cka_val_accs = get_val_accs(cka_data)
cka_ckas = get_ckas(cka_data)

l1_data = load_data(f"_in-784_hd-512-512_out-10_dr-0.5_strategy-iterative_type-l1.pkl")
l1_val_accs = get_val_accs(l1_data)
l1_ckas = get_ckas(l1_data)

In [295]:
cka_neurons = get_neurons(cka_data)
l1_neurons = get_neurons(l1_data)


print(len(cka_neurons[0]))

l0_all =[]
l1_all = []
for seed_cka_neurons, seed_l1_neurons in zip(cka_neurons, l1_neurons):
    layer0 = []
    layer1 = []

    ckap, l1p = [], []
    for cka_n, l1_n in zip(seed_cka_neurons[0], seed_l1_neurons[0]): 
        # print(cka_n)
        ckap.extend(cka_n)
        l1p.extend(l1_n)
        layer0.append(len(set(ckap).intersection(l1p)))
    
    ckap, l1p = [], []
    for cka_n, l1_n in zip(seed_cka_neurons[1], seed_l1_neurons[1]): 
        ckap.extend(cka_n)
        l1p.extend(l1_n)
        layer1.append(len(set(ckap).intersection(l1p)))

    l0_all.append(layer0)
    l1_all.append(layer1)


l0_all = np.array(l0_all)
l1_all = np.array(l1_all)
print(l0_all)

l0_all += l1_all


l0_all = np.array(l0_all, dtype=float)

pruned_prop = 0
for i in range(15):
    pruned_prop += (0.2) * (0.8 ** (i - 1))
    l0_all[:, i] /= 2 * pruned_prop * 512

print(l0_all)

for i in range(15):
    print(i, l0_all[:, i].mean(), l0_all[:, i].max(), l0_all[:, i].min())

# print(l0_all)
# print(l1_all)

2
[[ 55 108 166 219 262 311 349 382 406 424 436 451 463 472 480]
 [ 56 110 160 215 260 300 337 370 397 422 439 451 461 472 479]
 [ 55 109 158 214 257 297 334 364 392 416 432 446 460 471 479]
 [ 49 107 165 212 257 304 341 374 403 424 438 450 461 472 480]
 [ 62 116 169 218 264 308 349 376 405 422 440 453 461 470 477]
 [ 56 110 168 219 259 305 340 372 395 420 440 454 464 473 480]
 [ 64 108 167 213 260 299 333 368 397 420 435 453 464 475 481]
 [ 51 105 157 211 263 299 337 373 402 424 441 454 463 474 482]
 [ 56 106 163 208 258 304 343 368 396 420 436 451 463 472 480]]
[[0.421875   0.49913194 0.53630891 0.59943471 0.62516733 0.66599208
  0.6910088  0.70969147 0.72549996 0.73869138 0.74526804 0.75584771
  0.76221575 0.76979337 0.77492137]
 [0.41796875 0.49696181 0.54591445 0.58355564 0.61587116 0.65116872
  0.67321458 0.69279406 0.71106215 0.73344002 0.74526804 0.75500881
  0.76221575 0.77224494 0.77735059]
 [0.4375     0.50998264 0.53951076 0.5848789  0.61354712 0.64587467
  0.67420315 0.690

In [ ]:
s = ""
lines = s.split("\n")
for line in lines:
    d = line.split(" ")
    print(d[0], d[1], float(d[2]) - float(d[1]), float(d[1]) - float(d[3]))

In [296]:
# mean_cka_acc = cka_val_accs.mean(axis=0)
# mean_l1_acc = l1_val_accs.mean(axis=0)

# max_cka_acc = np.abs(mean_cka_acc - cka_val_accs.max(axis=0))
# max_l1_acc = np.abs(mean_l1_acc - l1_val_accs.max(axis=0))

# min_cka_acc = np.abs(mean_cka_acc - cka_val_accs.min(axis=0))
# min_l1_acc = np.abs(mean_l1_acc - l1_val_accs.min(axis=0))

# x = [0.2 * (0.8) ** (i - 1) for i in range(1, mean_cka_acc.shape[0] + 1)]
# for i in range(1, len(x)):
#     x[i] += x[i-1]
# # x = [np.round(v, 2) for v in x]
# x = [1 - v for v in x]

# print(x[::2])
# print()

# for i in range(mean_cka_acc.shape[0]):
#     print(int(np.round(x[i] * size)), mean_cka_acc[i], max_cka_acc[i], min_cka_acc[i])
# print()

# for i in range(mean_cka_acc.shape[0]):
#     print(int(np.round(x[i] * size)), mean_l1_acc[i], max_l1_acc[i], min_l1_acc[i])

# # std_cka_acc = cka_val_accs.std(axis=0)
# # std_l1_acc = l1_val_accs.std(axis=0)

# # print(len(cka_data[0][0]['prune'][0]['neurons']))

# # mean_cka_cka = cka_ckas.mean(axis=0)
# # mean_l1_cka = l1_ckas.mean(axis=0)

In [297]:
mean_cka_acc = cka_val_accs.mean(axis=0)
mean_l1_acc = l1_val_accs.mean(axis=0)

max_cka_acc = np.abs(mean_cka_acc - cka_val_accs.max(axis=0))
max_l1_acc = np.abs(mean_l1_acc - l1_val_accs.max(axis=0))

min_cka_acc = np.abs(mean_cka_acc - cka_val_accs.min(axis=0))
min_l1_acc = np.abs(mean_l1_acc - l1_val_accs.min(axis=0))

x = [0.1 * (0.9) ** (i - 1) for i in range(1, mean_cka_acc.shape[0] + 1)]
for i in range(1, len(x)):
    x[i] += x[i-1]
x = [np.round(v, 2) for v in x]
print(x)
x = x[::4]

print(x)
print()

for i in range(mean_cka_acc.shape[0]):
    print(i, mean_cka_acc[i], max_cka_acc[i], min_cka_acc[i])
print()

for i in range(mean_cka_acc.shape[0]):
    print(i, mean_l1_acc[i], max_l1_acc[i], min_l1_acc[i])

# std_cka_acc = cka_val_accs.std(axis=0)
# std_l1_acc = l1_val_accs.std(axis=0)

# print(len(cka_data[0][0]['prune'][0]['neurons']))

# mean_cka_cka = cka_ckas.mean(axis=0)
# mean_l1_cka = l1_ckas.mean(axis=0)

[0.1, 0.19, 0.27, 0.34, 0.41, 0.47, 0.52, 0.57, 0.61, 0.65, 0.69, 0.72, 0.75, 0.77, 0.79]
[0.1, 0.41, 0.61, 0.75]

0 0.9832111111111109 0.001388888888889106 0.0011111111111109517
1 0.9825555555555555 0.0014444444444444704 0.0018555555555554992
2 0.9817222222222222 0.0005777777777777882 0.00042222222222221273
3 0.9808777777777777 0.0007222222222222907 0.001277777777777711
4 0.9791777777777777 0.0012222222222223467 0.001677777777777667
5 0.9772000000000001 0.0007999999999999119 0.0007000000000000339
6 0.9753555555555555 0.0017444444444444374 0.0012555555555555653
7 0.9723555555555554 0.0012444444444446034 0.0014555555555554323
8 0.9686666666666667 0.0009333333333333416 0.0009666666666666712
9 0.9631777777777777 0.0020222222222222586 0.002677777777777668
10 0.9566888888888889 0.0022111111111110526 0.002888888888888941
11 0.9489000000000001 0.0032999999999999696 0.0029000000000001247
12 0.9387666666666666 0.0029333333333333433 0.003566666666666607
13 0.9270333333333333 0.00446666666666673 

In [298]:

# x = list(range(len(mean_l1_val_acc)))
# yerr = [
#     np.abs(mean_l1_val_acc - l1_val_accs.min(axis=0)), 
#     np.abs(mean_l1_val_acc - l1_val_accs.max(axis=0)),
# ]
# plt.errorbar(x, mean_l1_val_acc, yerr=yerr, label='l1')
# yerr = [
#     np.abs(mean_cka_val_acc - cka_val_accs.min(axis=0)), 
#     np.abs(mean_cka_val_acc - cka_val_accs.max(axis=0)),
# ]
# plt.errorbar(x, mean_cka_val_acc, yerr=yerr, label='cka')

# plt.title("Iterative pruning with L1 and CKA")
# plt.xlabel("Proportion of neurons in fc0 and fc1 pruned")
# plt.ylabel("Validation accuracy")

# plt.legend()
# plt.show()

# plt.plot(x, mean_l1_cka, label='l1_cka')
# plt.plot(x, mean_cka_cka, label='cka_cka')
# plt.legend()
# plt.show()